In [5]:
! pip install langchain
! pip install langchain_openai
! pip install langchain_community
! pip install pymongo
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import json
import openai
import pymongo

from dotenv import dotenv_values
from openai import AzureOpenAI

env_name = "example.env" 
config = dotenv_values(env_name)

openai.api_type = config['openai_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']

client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint = openai.api_base
)

In [ ]:
# Connection string
mongo_conn = config['mongo_vcore_connection_string']
mongo_client = pymongo.MongoClient(mongo_conn)

# Database name
DATABASE_NAME = "ResturantChain"
db = mongo_client[DATABASE_NAME]

# Drop the database if it already exists (consider if this is really needed, as it will remove all existing data)
#mongo_client.drop_database(DATABASE_NAME)

# Collection names
collection_names = ["Contoso", "Contoso West", "Users"]

# Iterate through the collection names and create them if they do not exist
for collection_name in collection_names:
    if collection_name not in db.list_collection_names():
        # Creates a collection
        db.create_collection(collection_name)
        print(f"Created collection '{collection_name}'.")
    else:
        print(f"Using existing collection: '{collection_name}'.")

Langchain documentation code testing

In [6]:
from langchain.community.vectorstores.azure_cosmosdb import AzureCosmosDBVectorStore, AzureCosmosDBSimilarityType
from langchain.core.prompts import ChatPromptTemplate
from langchain.openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains.combine_documents import createStuffDocumentsChain
from langchain.chains.retrieval import createRetrievalChain
from langchain.document_loaders.fs.text import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents from file
loader = TextLoader("./data/state_of_the_union.txt")
raw_documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunkSize=1000, chunkOverlap=0)
documents = splitter.splitDocuments(raw_documents)

# Create Azure Cosmos DB vector store
store = AzureCosmosDBVectorStore.fromDocuments(
  documents,
  OpenAIEmbeddings(),
  databaseName="langchain",
  collectionName="documents"
)

# Create the index
numLists = 100
dimensions = 1536
similarity = AzureCosmosDBSimilarityType.COS
store.createIndex(numLists, dimensions, similarity)

# Performs a similarity search
result_documents = store.similaritySearch(
  "What did the president say about Ketanji Brown Jackson?"
)

print("Similarity search results:")
print(result_documents[0].pageContent)
'''
  Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

  Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

  One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

  And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
'''

# Use the store as part of a chain
model = ChatOpenAI(model="gpt-3.5-turbo-1106")
question_answering_prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "Answer the user's questions based on the below context:\n\n{context}",
  ],
  ["human", "{input}"],
])

combine_docs_chain = createStuffDocumentsChain(llm=model, prompt=question_answering_prompt)

chain = createRetrievalChain(retriever=store.asRetriever(), combineDocsChain=combine_docs_chain)

res = chain.invoke(input="What is the president's top priority regarding prices?")

print("Chain response:")
print(res.answer)
'''
  The president's top priority is getting prices under control.
'''

# Clean up
store.delete()
store.close()


ModuleNotFoundError: No module named 'langchain.community'